# Figure 14 ignition delay times from LLNL for iso-octane in N2 as bath gas comparing to the Dagaut et al 2014 

In [1]:
import cantera as ct
import numpy as np
%matplotlib inline

from matplotlib import pyplot as plt

# Importing the GTL mechanism from Dagaut for Iso-octane and comparing to LLNL and RMG model

In [2]:
ct.__version__

'2.4.0'

## Using the model of Atef et al from KAUST, UCONN and NUI to get fuel and air composition in argon and nitrogen bath gases 

In [ ]:
from cantera import ck2cti
ck2cti.main([
    '--input=/Users/ebubeajulu/Code/GTL/ic8/Atef_mech/ic8-detailed-mechanism-CNF-revised.txt',
    '--thermo=/Users/ebubeajulu/Code/GTL/ic8/Atef_mech/updated_therm.txt',
    '--output=/Users/ebubeajulu/Code/GTL/ic8/Atef_mech/ic8-detailed-mechanism-CNF-revised.cti',
    '--permissive'])

In [ ]:
gas = ct.Solution('/Users/ebubeajulu/Code/GTL/Naik_CNF2010_alternateJet_mechanism.cti')

In [4]:
sorted(gas.species_names)

['A(OOH)ETRAA',
 'A(OOH)ETRAB',
 'A(OOH)ETRAC',
 'A(OOH)ETRAD',
 'A(OOH)ETRBC',
 'A(OOH)ETRBD',
 'A(OOH)ETRCD',
 'A(OOH)ETRDD',
 'A-AC5H10O',
 'A-AC6H12O',
 'A-BC5H10O',
 'A-BC6H12O',
 'A-CC5H10O',
 'A-CC6H12O',
 'A-DC5H10O',
 'A-DC6H12O',
 'A-EC6H12O',
 'AC3H4CH2CHO',
 'AC3H4CH2COCH3',
 'AC3H4COC2H5',
 'AC3H4COCH3',
 'AC3H5CHCHO',
 'AC3H5CHCOCH3',
 'AC3H5CHO',
 'AC3H5CO',
 'AC3H5OCH2',
 'AC3H5OOH',
 'AC4H7OOH',
 'AC5H10',
 'AC5H10CHO',
 'AC5H10OH',
 'AC5H10OOH-A',
 'AC5H10OOH-AO2',
 'AC5H10OOH-B',
 'AC5H10OOH-BO2',
 'AC5H10OOH-C',
 'AC5H10OOH-CO2',
 'AC5H10OOH-D',
 'AC5H10OOH-DO2',
 'AC5H11',
 'AC5H11O',
 'AC5H11O2',
 'AC5H11O2H',
 'AC5H9-A1',
 'AC5H9-A2',
 'AC5H9-C',
 'AC5H9-D',
 'AC5H9O-A2',
 'AC5H9O-C',
 'AC6H11-A2',
 'AC6H11-C',
 'AC6H11-D',
 'AC6H11-E',
 'AC6H11O-C',
 'AC6H12',
 'AC6H12CHO-B',
 'AC6H12CHO-D',
 'AC6H12OH',
 'AC6H12OOH-A',
 'AC6H12OOH-AO2',
 'AC6H12OOH-B',
 'AC6H12OOH-BO2',
 'AC6H12OOH-C',
 'AC6H12OOH-CO2',
 'AC6H12OOH-D',
 'AC6H12OOH-DO2',
 'AC6H12OOH-E',
 'AC6H12

In [5]:
for species in gas.species():
    if species.composition == {'C':8, 'H':18}:
        print(species.name)

IC8H18
NC8H18
C8H18-2


In [8]:
gas.TP = 900, 10e5
gas.set_equivalence_ratio(phi=1.0, fuel='IC8H18', oxidizer='O2:0.21, N2:0.158, Ar:0.632')
gas()



  gas:

       temperature             900  K
          pressure           1e+06  Pa
           density         5.03528  kg/m^3
  mean mol. weight         37.6791  amu

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy      3.7523e+05        1.414e+07     J
   internal energy      1.7663e+05        6.655e+06     J
           entropy            5219        1.966e+05     J/K
    Gibbs function     -4.3219e+06       -1.628e+08     J
 heat capacity c_p          850.66        3.205e+04     J/K
 heat capacity c_v          629.99        2.374e+04     J/K

                           X                 Y          Chem. Pot. / RT
                     -------------     ------------     ------------
                AR       0.621558         0.658986            -17.9
                N2       0.155389         0.115528         -24.1696
                O2        0.20653         0.175395         -25.5742
            IC8H18      0.01

In [ ]:
def set_gas_concentrations(gas, phi):
    """
    Set the concentrations in the gas according to the description
    in the paper by 
    """
    # Set it to desired stoichiometry with synthetic air (20% O2 : 80% N2)
    gas.set_equivalence_ratio(phi=phi, fuel='ic8h18', oxidizer='o2:0.2, n2:0.8' )
    # Then dilute the whole thing 1:2 with extra N2
    X = gas.X / 3.0 
    X[gas.species_index('n2')] += 2./3.
    gas.X = X 
    
# We divide the total gas mole fractions by 3 since the ratio of dilution of fuel in the 
# synthetic air oxygen to nitrogen is 1:2. Also since the total amount of nitrogen is now 2/3 based on the 
# dilution ratio, we add 2/3 to the gas mole fraction of N2 and then make the gas mole fraction equal to the 
# new value of the dilution X 

set_gas_concentrations(gas, 1.0)
gas()

In [ ]:
def get_ignition_delay(gas, temperature, pressure_bar, mole_fractions, plot=False):
    """
    A general function to find the igniton delay.
    Using C2H + O --> CH* as the indication of ignition
    
    gas is a cantera Solution object
    temperature in K
    pressure_bar is in bar
    mole_fractions is a dict
    """
    gas.TPX = temperature, pressure_bar*1e5, mole_fractions

    reactor = ct.IdealGasReactor(gas)
    reactor_network = ct.ReactorNet([reactor])
    
    time = 0.0
    end_time = 100e-3
    
    times = []
    concentrations = []
    pressures = []
    temperatures = []
    
    print_data = True
    while time < end_time:
        time = reactor_network.time
        times.append(time)
        temperatures.append(reactor.T)
        pressures.append(reactor.thermo.P)
        concentrations.append(reactor.thermo.concentrations)
        # take a timestep
        # the size of the step will be determined by the ODE solver
        # depending on how quickly things are changing.
        reactor_network.step()
    
    print("Reached end time {0:.2f} ms in {1} steps".format(times[-1]*1e3, len(times)))
    # convert the lists into arrays
    concentrations = np.array(concentrations)
    times = np.array(times)
    pressures = np.array(pressures)
    temperatures = np.array(temperatures)

    if plot:
        plt.subplot(2,1,1)
        plt.plot(times*1e3, pressures/1e5)
        plt.ylabel("Pressure (bar)", color='b')
        ax2 = plt.gca().twinx()
        ax2.set_ylabel('Temperature (K)', color='r')
        ax2.plot(times*1e3, temperatures, 'r')
    # Using C2H + O --> CH* as the indication of ignition
    i_c2h = gas.species_index('C2H')   
    i_o = gas.species_index('O')
    excited_ch_generation = concentrations[:,i_o] * concentrations[:,i_c2h]
    if plot:
        plt.subplot(2,1,2)
        plt.plot(times*1e3, excited_ch_generation, 'g')
        plt.ylabel("CH* emission")
        plt.ylim(0,max(1e-13,1.1*max(excited_ch_generation)))
        plt.xlabel("Time (ms)")
        plt.tight_layout()
        plt.show()
    step_with_highest_ch_gen = excited_ch_generation.argmax()
    if step_with_highest_ch_gen > 1 and excited_ch_generation.max()>1e-20:
        ignition_time_ms = 1e3 * times[step_with_highest_ch_gen]
        print("At {0} K {1} bar, ignition delay time is {2} ms".format(temperature, pressure_bar, ignition_time_ms))
        return ignition_time_ms
    else:
        print("At {0} K {1} bar, no ignition detected".format(temperature, pressure_bar))
        return np.infty


In [ ]:
def get_ignition_delay_air_n2(temperature, pressure_bar=16, phi=1.0, plot=False):
    """
    For figure 14
    
    temperature in K
    pressure in bar
    phi is equivalence ratio
    """
    gas.TP = temperature, pressure_bar*1e5
    set_gas_concentrations(gas, phi)
    mole_fractions = gas.X
    
    if plot:
        gas()

    time = get_ignition_delay(gas, temperature, pressure_bar, mole_fractions, plot=plot)
    return time

In [ ]:
get_ignition_delay_air_n2(1000/0.8, 16, 1, plot=True)
plt.rcParams['figure.figsize'] = [10, 8]


In [ ]:
DAG_temperatures = 1000/np.linspace(0.70,1.2,25)
pressures_bar = 16
phi = 1

DAG_results = dict()
times = []
for T in DAG_temperatures:
    t = get_ignition_delay_air_n2(T,pressures_bar, phi, plot=False)
    times.append(t)
    DAG_results[pressures_bar] = times

In [ ]:
plt.semilogy(1000/DAG_temperatures,times,label='Dagaut et al model without moving wall {}bar'.format(pressures_bar))
plt.legend(loc='best')
plt.xlabel('1000 K / $T$')
plt.ylabel('$\\tau_{ign} = t([CH^*]_{max})/ms$')
plt.rcParams['figure.figsize'] = [10, 8]



In [ ]:
import os
filename = "/Users/ebubeajulu/Code/GTL/ic8_ignition_delay_Dagaut_without_moving_wall.pkl"
os.makedirs(os.path.dirname(filename), exist_ok=True)

with open('ic8_ignition_delay_Dagaut_without_moving_wall.pkl', 'wb') as f:
    pickle.dump((DAG_temperatures, DAG_results[pressures_bar]), f)

In [ ]:
import pickle
with open('ic8-LLNL.pkl','rb') as fp:
    (LLNL_temperatures, LLNL_results) = pickle.load(fp)

In [ ]:
import pickle
with open('ic8-RMG.pkl','rb') as fp:
    (RMG_temperatures, RMG_results) = pickle.load(fp)

In [ ]:
predict_ic8 = """0.657614                 0.0438935
0.660053                 0.0508193
0.668666                 0.0605986
0.674810                 0.0712038
0.682193                 0.0824521
0.690803                 0.0997690
0.695710                 0.117224
0.705556                 0.141850
0.712933                 0.169140
0.722793                 0.193028
0.730166                 0.233559
0.740019                 0.274467
0.748628                 0.332111
0.759721                 0.384622
0.765868                 0.445364
0.775710                 0.546873
0.786810                 0.615061
0.799138                 0.712339
0.806515                 0.849383
0.818843                 0.983721
0.829936                 1.13926
0.843501                 1.31950
0.852120                 1.52800
0.864449                 1.76967
0.878014                 2.04964
0.889110                 2.33921
0.900212                 2.59264
0.910061                 3.09168
0.927338                 3.52918
0.935965                 3.96890
0.942116                 4.52889
0.958153                 5.24581
0.969245                 6.07524
0.985293                 6.73451
0.993909                 7.91373
1.00872                  8.90155
1.02476                  9.86753
1.03338                  11.4268
1.04572                  12.8521
1.06423                  14.6714
1.07409                  16.9904
1.08520                  18.5574
1.10124                  21.1825
1.11357                  23.4784
1.12466                  27.5917
1.13824                  30.1389
1.14933                  35.4191
1.16290                  39.8386
1.17523                  46.1394
1.18633                  51.8925
1.19865                  61.8858
1.20728                  68.5850
1.21961                  80.6040
1.23070                  94.7255
"""

ign_times = []
temps = []

for k in predict_ic8.splitlines():
    temp, ign_time = k.split()
    temps.append(float(temp))
    ign_times.append(float(ign_time))
    
ign_times = np.array(ign_times)
temps = np.array(temps)

In [ ]:
experiment_ic8 = """ 0.742197            0.129438
0.792731            0.238540
0.858775            0.582623
0.897119            1.02386
0.916046            1.35725
1.01424             4.07608
1.05446             8.48794
1.08368             10.3738
1.11478             15.2007
1.16585             24.9217
1.17410             28.0282
1.18433             30.0848
"""

exp_ign_times = []
exp_temps = []



for z in experiment_ic8.splitlines():
    exp_temp, exp_ign_time = z.split()
    exp_temps.append(float(exp_temp))
    exp_ign_times.append(float(exp_ign_time))
    
exp_ign_times = np.array(exp_ign_times)
exp_temps = np.array(exp_temps)



In [ ]:
plt.rcParams.update({'font.size': 15})

# plt.semilogy(1000/LLNL_temperatures,LLNL_results[pressures_bar],label='LLNL {}bar'.format(pressures_bar))
# plt.legend(loc='best')
# plt.xlabel('1000 K / $T$')
# plt.ylabel('$\\tau_{ign} = t([CH^*]_{max})/ms$')


# # plt.semilogy(1000/RMG_temperatures,RMG_results[pressures_bar],label='RMG {}bar'.format(pressures_bar))
# # plt.legend(loc='best')
# # plt.xlabel('1000 K / $T$')
# # plt.ylabel('$\\tau_{ign} = t([CH^*]_{max})/ms$')

plt.semilogy(temps,ign_times, 'r.:',label='Dagaut et al. Prediction Fig.14'.format(pressures_bar))

plt.semilogy(exp_temps,exp_ign_times, 'g+', label='Dagaut et al. Experiment Fig.14'.format(pressures_bar))

plt.semilogy(1000/DAG_temperatures,times,label='Dagaut et Al without moving wall {} bar'.format(pressures_bar))
plt.legend(loc='best')
plt.xlabel('1000 K / $T$')
plt.ylabel('$\\tau_{ign} = t([CH^*]_{max})/ms$')
plt.rcParams['figure.figsize'] = [15, 10]

plt.show